In [1]:
from src.features.featurizer import Featurizer2
import polars as pl

In [2]:
featurizer = Featurizer2()


In [3]:
print(featurizer.raw_data_paths)
print(featurizer.symbols)
print(featurizer.schemas)

['xnas-itch-20180501-20250314.ohlcv-1m.parquet', 'xnas-itch-20180501-20250314.bbo-1m.parquet', 'xnas-itch-20180501-20250314.imbalance.parquet']
['AAPL', 'AMZN', 'META', 'TSLA', 'NFLX', 'MSFT', 'AVGO', 'ADBE', 'IBM', 'GOOGL', 'NVDA', 'INTC', 'AMD']
['ohlcv-1m', 'bbo-1m', 'imbalance']


In [4]:
ohlcv_data, bbo_data, imbalance_data = featurizer.process()

time_index shape: (10_029_511, 5)
┌─────────────────────────┬───────┬──────────────┬───────────────┬────────┐
│ ts_event                ┆ rtype ┆ publisher_id ┆ instrument_id ┆ symbol │
│ ---                     ┆ ---   ┆ ---          ┆ ---           ┆ ---    │
│ datetime[ns, UTC]       ┆ u8    ┆ u16          ┆ u32           ┆ str    │
╞═════════════════════════╪═══════╪══════════════╪═══════════════╪════════╡
│ 2018-05-01 08:11:00 UTC ┆ 33    ┆ 2            ┆ 3363          ┆ GOOGL  │
│ 2018-05-01 08:27:00 UTC ┆ 33    ┆ 2            ┆ 13            ┆ AAPL   │
│ 2018-05-01 08:30:00 UTC ┆ 33    ┆ 2            ┆ 5156          ┆ MSFT   │
│ 2018-05-01 08:35:00 UTC ┆ 33    ┆ 2            ┆ 13            ┆ AAPL   │
│ 2018-05-01 08:38:00 UTC ┆ 33    ┆ 2            ┆ 13            ┆ AAPL   │
│ …                       ┆ …     ┆ …            ┆ …             ┆ …      │
│ 2025-03-14 23:59:00 UTC ┆ 33    ┆ 2            ┆ 1429          ┆ AVGO   │
│ 2025-03-14 23:59:00 UTC ┆ 33    ┆ 2            ┆ 108

In [17]:
imbalance_data.filter(pl.col("symbol") == "AMD")

ts_event,rtype,publisher_id,instrument_id,ref_price,auction_time,cont_book_clr_price,auct_interest_clr_price,ssr_filling_price,ind_match_price,upper_collar,lower_collar,paired_qty,total_imbalance_qty,market_imbalance_qty,unpaired_qty,auction_type,side,auction_status,freeze_status,num_extensions,unpaired_side,significant_imbalance,symbol,ts_recv
"datetime[ns, UTC]",u8,u16,u32,f64,u64,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,str,str,u8,u8,u8,str,str,str,"datetime[ns, UTC]"


In [56]:
path1 = "data/raw/xnas-itch-20180501-20250314.imbalance.parquet"

imbalance_data_1 = pl.read_parquet(path1)

In [57]:
import os 

directory = "data/cache"
keyword = "imbalance"

matching_files = [
        f for f in os.listdir(directory)
        if os.path.isfile(os.path.join(directory, f)) and keyword in f
    ]
    


In [ ]:
for matching_file in matching_files:
    print(matching_file)
    data = pl.read_parquet(os.path.join(directory, matching_file))
    data.drop_in_place("ts_recv")
    print(imbalance_data_1.columns)
    print(data.columns)
    imbalance_data_1.drop_in_place("ts_recv")
    data = data.select([ "ts_event" ]+ [col for col in data.columns if col != "ts_event"])
    imbalance_data_1 = imbalance_data_1.select([ "ts_event" ]+ [col for col in bbo_data_1.columns if col != "ts_event"])
    imbalance_data_1 = pl.concat([imbalance_data_1 ,data])


MSFT_imbalance.parquet
['ts_event', 'rtype', 'publisher_id', 'instrument_id', 'ref_price', 'auction_time', 'cont_book_clr_price', 'auct_interest_clr_price', 'ssr_filling_price', 'ind_match_price', 'upper_collar', 'lower_collar', 'paired_qty', 'total_imbalance_qty', 'market_imbalance_qty', 'unpaired_qty', 'auction_type', 'side', 'auction_status', 'freeze_status', 'num_extensions', 'unpaired_side', 'significant_imbalance', 'symbol', 'ts_recv']
['ts_event', 'rtype', 'publisher_id', 'instrument_id', 'ref_price', 'auction_time', 'cont_book_clr_price', 'auct_interest_clr_price', 'ssr_filling_price', 'ind_match_price', 'upper_collar', 'lower_collar', 'paired_qty', 'total_imbalance_qty', 'market_imbalance_qty', 'unpaired_qty', 'auction_type', 'side', 'auction_status', 'freeze_status', 'num_extensions', 'unpaired_side', 'significant_imbalance', 'symbol']


ShapeError: unable to append to a DataFrame of width 25 with a DataFrame of width 24

In [59]:
imbalance_data_1.sort("ts_event")

ts_event,rtype,publisher_id,instrument_id,ref_price,auction_time,cont_book_clr_price,auct_interest_clr_price,ssr_filling_price,ind_match_price,upper_collar,lower_collar,paired_qty,total_imbalance_qty,market_imbalance_qty,unpaired_qty,auction_type,side,auction_status,freeze_status,num_extensions,unpaired_side,significant_imbalance,symbol,ts_recv
"datetime[ns, UTC]",u8,u16,u32,f64,u64,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,str,str,u8,u8,u8,str,str,str,"datetime[ns, UTC]"
2018-05-01 13:28:00.029946658 UTC,20,2,89,220.68,0,220.68,220.68,0.0,0.0,0.0,0.0,26422,0,0,0,"""O""","""N""",0,0,0,"""N""","""L""","""ADBE""",2018-05-01 13:28:00.029957599 UTC
2018-05-01 13:28:00.068274053 UTC,20,2,5156,93.23,0,93.16,93.16,0.0,0.0,0.0,0.0,227040,10480,0,0,"""O""","""A""",0,0,0,"""N""","""L""","""MSFT""",2018-05-01 13:28:00.068284668 UTC
2018-05-01 13:28:00.073546234 UTC,20,2,601,229.92,0,229.92,229.92,0.0,0.0,0.0,0.0,28989,0,0,0,"""O""","""N""",0,0,0,"""N""","""L""","""AVGO""",2018-05-01 13:28:00.073557361 UTC
2018-05-01 13:28:00.087979785 UTC,20,2,375,1566.26,0,1562.73,1562.33,0.0,0.0,0.0,0.0,31124,15413,0,0,"""O""","""A""",0,0,0,"""N""","""L""","""AMZN""",2018-05-01 13:28:00.087991394 UTC
2018-05-01 13:28:00.088650077 UTC,20,2,5367,310.99,0,310.57,310.57,0.0,0.0,0.0,0.0,41760,9772,0,0,"""O""","""A""",0,0,0,"""N""","""L""","""NFLX""",2018-05-01 13:28:00.088660620 UTC
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-03-14 20:05:00.024225436 UTC,20,2,11275,918.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,"""A""","""N""",0,0,0,"""N""","""~""","""NFLX""",2025-03-14 20:05:00.024391280 UTC
2025-03-14 20:05:00.024315038 UTC,20,2,16244,249.98,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,"""A""","""N""",0,0,0,"""N""","""~""","""TSLA""",2025-03-14 20:05:00.024480892 UTC
2025-03-14 20:05:00.025602266 UTC,20,2,38,213.49,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,"""A""","""N""",0,0,0,"""N""","""~""","""AAPL""",2025-03-14 20:05:00.025768363 UTC


In [ ]:
imbalance_data_1.filter(pl.col("symbol") == "AMD")

ts_event,rtype,publisher_id,instrument_id,side,price,size,flags,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol
"datetime[ns, UTC]",u8,u16,u32,str,f64,u32,u8,u32,f64,f64,u32,u32,u32,u32,str
null,196,2,328,"""N""",null,0,130,1969559,10.71,null,300,0,1,0,"""AMD"""
null,196,2,328,"""N""",null,0,130,1994591,10.71,10.87,300,21,1,1,"""AMD"""
null,196,2,328,"""N""",null,0,130,2021701,0.01,199999.99,100,100,1,1,"""AMD"""
null,196,2,328,"""N""",null,0,130,2024655,10.85,13.7,300,9600,1,1,"""AMD"""
null,196,2,328,"""N""",null,0,130,2172755,0.01,13.7,100,9600,1,1,"""AMD"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-03-14 23:51:21.743318674 UTC,196,2,773,"""N""",100.93,5,130,707745162,100.84,100.98,9,1,2,1,"""AMD"""
2025-03-14 23:51:21.743318674 UTC,196,2,773,"""N""",100.93,5,130,707764088,100.84,101.0,9,100,2,1,"""AMD"""
2025-03-14 23:51:21.743318674 UTC,196,2,773,"""N""",100.93,5,130,707766338,100.84,101.0,8,100,1,1,"""AMD"""


In [52]:
bbo_data_1.write_parquet(path1)